In [46]:
import random
import csv
import pandas as pd
import dataframe_image as dfi

In [3]:
# write states to .csv
def save_states(states):
    with open('states.csv', mode='w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['States'])

        for state in states:
            csv_writer.writerow([state])

In [4]:
# write expected moves to .csv
def save_moves(moves):
    with open('moves.csv', mode='w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['No of Moves', 'Probability'])

        for move in moves:
            csv_writer.writerow([move, moves[move]])

In [14]:
def generate_tile():
    # probability of getting a 2 = 0.9, probability of getting a 4 = 0.1
    return (2 if random.random() < 0.9 else 4)


In [15]:
def merge_tiles(state):
    new_state = []
    # merging
    i = 0
    while i < len(state):
        if i < len(state) - 1 and state[i] == state[i+1]:
            new_state.append(state[i] * 2)
            i += 2
        else:
            new_state.append(state[i])
            i += 1
    return new_state

In [29]:
def generate_states(states, cur_state):
    if cur_state[-1] == 2048:
        return
    else:
        # merge first
        cur_state = merge_tiles(cur_state)
        
        # generate a tile
        next_state_with_2 = cur_state + [2]
        next_state_with_2.sort()
        next_state_with_4 = cur_state + [4]
        next_state_with_4.sort()


        if next_state_with_2 not in states:
            states.append(next_state_with_2)
            generate_states(states, next_state_with_2)
        if next_state_with_4 not in states:
            states.append(next_state_with_4)
            generate_states(states, next_state_with_4)

In [30]:
states = [[0], [2, 2], [2, 4], [4, 4]]
initial_states = [[2, 2], [2, 4], [4, 4]]

for initial_state in initial_states:
    generate_states(states, initial_state)

print(states)

[[0], [2, 2], [2, 4], [4, 4], [2, 2, 4], [2, 4, 4], [2, 2, 8], [2, 4, 8], [2, 2, 4, 8], [2, 4, 4, 8], [2, 2, 8, 8], [2, 4, 16], [2, 2, 4, 16], [2, 4, 4, 16], [2, 2, 8, 16], [2, 4, 8, 16], [2, 2, 4, 8, 16], [2, 4, 4, 8, 16], [2, 2, 8, 8, 16], [2, 4, 16, 16], [2, 2, 4, 32], [2, 4, 4, 32], [2, 2, 8, 32], [2, 4, 8, 32], [2, 2, 4, 8, 32], [2, 4, 4, 8, 32], [2, 2, 8, 8, 32], [2, 4, 16, 32], [2, 2, 4, 16, 32], [2, 4, 4, 16, 32], [2, 2, 8, 16, 32], [2, 4, 8, 16, 32], [2, 2, 4, 8, 16, 32], [2, 4, 4, 8, 16, 32], [2, 2, 8, 8, 16, 32], [2, 4, 16, 16, 32], [2, 2, 4, 32, 32], [2, 4, 4, 64], [2, 2, 8, 64], [2, 4, 8, 64], [2, 2, 4, 8, 64], [2, 4, 4, 8, 64], [2, 2, 8, 8, 64], [2, 4, 16, 64], [2, 2, 4, 16, 64], [2, 4, 4, 16, 64], [2, 2, 8, 16, 64], [2, 4, 8, 16, 64], [2, 2, 4, 8, 16, 64], [2, 4, 4, 8, 16, 64], [2, 2, 8, 8, 16, 64], [2, 4, 16, 16, 64], [2, 2, 4, 32, 64], [2, 4, 4, 32, 64], [2, 2, 8, 32, 64], [2, 4, 8, 32, 64], [2, 2, 4, 8, 32, 64], [2, 4, 4, 8, 32, 64], [2, 2, 8, 8, 32, 64], [2, 4, 16, 3

In [57]:
# doing things on states
states.sort(key=sum)

states_data = {
    'state': states,
    'sum': [sum(state) for state in states]
}

states_df = pd.DataFrame(states_data)
display(states_df)
states_df.to_csv("states.csv")

,state,sum
0,[0],0
1,"[2, 2]",4
2,"[2, 4]",6
3,"[2, 2, 4]",8
4,"[4, 4]",8
...,...,...
3482,"[4, 8, 8, 16, 1024, 1024]",2084
3483,"[4, 8, 8, 16, 2048]",2084
3484,"[2, 4, 8, 8, 16, 2048]",2086
3485,"[2, 4, 16, 16, 2048]",2086


In [50]:
# absorbing states
absorbing_states = [state for state in states if state[-1] == 2048]

,state,sum
0,"[2, 4, 4, 8, 2048]",2066
1,"[2, 2, 8, 8, 2048]",2068
2,"[2, 2, 16, 2048]",2068
3,"[4, 4, 4, 8, 2048]",2068
4,"[2, 4, 8, 8, 2048]",2070
5,"[2, 4, 16, 2048]",2070
6,"[2, 2, 4, 16, 2048]",2072
7,"[4, 4, 8, 8, 2048]",2072
8,"[4, 4, 16, 2048]",2072
9,"[2, 4, 4, 16, 2048]",2074


In [67]:
def simulate():
    state = [0]
    no_of_moves = 0
    while state[-1] != 2048:
        if len(state) == 1:
            # generate two random tiles
            state = []
            state.append(generate_tile())
            state.append(generate_tile())
        else:
            # merging
            state = merge_tiles(state) 

            # generate a new cell
            state.append(generate_tile())

        # disregard order
        state.sort()

        no_of_moves += 1
    absorbing_probabilities[tuple(state)] += 1
    return no_of_moves




no_of_trials = 100000
absorbing_probabilities = {tuple(absorbing_state): 0 for absorbing_state in absorbing_states}
moves = {}

for _ in range(no_of_trials):
    move = simulate()
    
    if move in moves:
        moves[move] += 1
    else:
        moves[move] = 1

for item in absorbing_probabilities:
    absorbing_probabilities[tuple(item)] /= no_of_trials


In [66]:
# absorbing probabilities
assert(sum(absorbing_probabilities.values()) == 1.0, "wrong probability")
absorbing_states_data = {
    'absorbing state': absorbing_states,
    'absorbing probability': [absorbing_probabilities[tuple(state)] for state in absorbing_states],
    'sum': [sum(state) for state in absorbing_states]
}

absorbing_states_df = pd.DataFrame(absorbing_states_data)
display(absorbing_states_df)
absorbing_states_df.to_csv("absorbing_states.csv")

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/gm/cpf51y7d6xlg6598mnnmhg680000gn/T/ipykernel_6486/1354872188.py:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(sum(absorbing_probabilities.values()) == 1.0, "wrong probability")


,absorbing state,absorbing probability,sum
0,"[2, 4, 4, 8, 2048]",0.035133,2066
1,"[2, 2, 8, 8, 2048]",0.348933,2068
2,"[2, 2, 16, 2048]",0.004167,2068
3,"[4, 4, 4, 8, 2048]",0.003867,2068
4,"[2, 4, 8, 8, 2048]",0.077800,2070
5,"[2, 4, 16, 2048]",0.285133,2070
6,"[2, 2, 4, 16, 2048]",0.142933,2072
7,"[4, 4, 8, 8, 2048]",0.003767,2072
8,"[4, 4, 16, 2048]",0.034300,2072
9,"[2, 4, 4, 16, 2048]",0.041267,2074
